# TODO fase 1
 - calculate predictions for a stock fx apple
 - statistics for the predictions: distribution of errors, percentage prediction with the right direction:  close>open, open<close
 - visualize ohlc as bands and on stock curve
 - split in train, test so that test data are taken from time periodens after the training data and possibly from stocks that have not been used for training 

 - tjek quantile objective function
 - objectiv function that weigh open and closing prices higher

 - add day of week to training data
 - speed up training: improve the pandas part by 60 times
                                                                                                                                   
# TODO fase 2
 - language modelling

In [ ]:
#default_exp finance.training

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export
from lib.data.lists import *
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

from lib.finance.data import *

# create stock like dataframe with generated data
Multiindex with stock, date and a column of dat generate by a function such as sin 


# Prepare data for prediction of next days stock prices the next day
Create OHLC price changes as percentages relative to the close of the previous day. Furthermore the ohlc is supplemented by the previous days closing.

# Create dataset

In [ ]:
# export
import torch
from lib.data.lists import *
from datetime import datetime, timedelta
from math import pi
from scipy import signal

def prepareFunctionData():
    stocks = ["A","B","C","D","E","F","G","H","I","J","K"]
    dates  = np.arange(datetime(2018,1,1), datetime(2019,1,1), timedelta(days=1)).astype(datetime)
    step=4
    x_signal = 2*pi/len(dates)*np.arange(0,len(dates)*step,step)
    #signal = np.sin(x_signal) + signal.square(2 * np.pi * 5 * t))
    y_signal = 1*np.sin(x_signal) + 0.125*signal.square(x_signal*8)
    #print(signal.square(x_signal)+y_signal)
    #print(y_signal)
    #print(x_signal)
    #print(signal)
    #print(len(stocks), len(dates), (len(signal)))

    stock_index=[]
    date_index=[]
    signal_data=[]
    for stock in stocks:
        stock_index.extend(np.repeat(stock,len(y_signal)))
        date_index.extend(dates)
        signal_data.extend(y_signal)
    tuples    = list(zip(stock_index,date_index))
    index     = pd.MultiIndex.from_tuples(tuples, names=[TICKER, DATE])
    df_prices = pd.DataFrame({OPEN:signal_data},index=index)
    return df_prices, stocks, stocks, []
df_prices, stocks, validStocks, inValidStocks = prepareFunctionData()
#df_prices.head()

In [ ]:
from lib.finance.lineplot import *
df =  df_prices.iloc[:]
dates = df.loc["A"].index
reference  = (df.loc["A",OPEN],"open reference","blue","line")
prediction = (df.loc["A",OPEN],"open prediction","orange","circle")
plotSignalVSPrediction(dates,[reference,prediction])


Loading BokehJS ...

In [ ]:
#statistics on the mature stocks
df_prices.loc[validStocks,[OPEN]].describe(percentiles=[0.0002, 0.25, 0.75, 0.9998])        

,Open
count,4015.000000
mean,0.000342
std,0.718872
min,-1.124917
0.02%,-1.124917
25%,-0.698772
50%,0.003748
75%,0.698772
99.98%,1.124917
max,1.124917


# Test

In [ ]:
%%time 
from lib.learner.learner import*
from lib.learner.optimizers import*
from lib.model.model import*
from lib.model.modelmanager import*
import torch.nn as nn

seq_length = 64
training_column_names=[OPEN]
ohlc_ds = OHLCDataset(df_prices, stocks=validStocks, column_names=training_column_names, seq_length=seq_length)
ohlc_ds.initializeSquenzing()

train_ds, test_ds = ohlc_ds.split2train_test(0.25)
print(f"number of stocks, train stocks, test stocks: {len(ohlc_ds.stocks)}, {len(train_ds.stocks)}, {len(test_ds.stocks)}")

databunch = DataBunch(train_ds.dataloader(batch_size=8, shuffle=True,  drop_last=True), \
                      test_ds.dataloader( batch_size=8, shuffle=False, drop_last=False), \
                      c_in=len(ohlc_ds.column_names), c_out=len(ohlc_ds.column_names))

tensor_input_view = view_tfm(len(ohlc_ds.column_names),seq_length)

databunch.c_in,databunch.c_out, 
b = next(iter(databunch.train_dl))
b= tensor_input_view(b[0])
#torch.Size([8, 16, 1])
#b.shape

SyntaxError: invalid syntax (model.py, line 230)

In [ ]:
%time l = [len(b[0]) for b in databunch.train_dl]
#next(iter(databunch.train_dl))

NameError: name 'databunch' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

input    = torch.randn(1,seq_length)
half_seq = int(seq_length/2.+.5)

model = nn.Sequential(
    nn.Linear(seq_length,seq_length),
    torch.nn.Tanh(), #nn.LeakyReLU(inplace=True), #F.relu(x, inplace=True),
    nn.Linear(seq_length,half_seq),
    torch.nn.Tanh(), #nn.LeakyReLU(inplace=True), #F.relu(x, inplace=True),
    nn.Linear(half_seq,half_seq),
    torch.nn.Tanh(), #nn.LeakyReLU(inplace=True), #F.relu(x, inplace=True),
    nn.Linear(half_seq,1),
    #nn.Flatten(0, 1)
    nn.Flatten()
)

#model(input).shape
model(b).shape

NameError: name 'seq_length' is not defined

In [ ]:
%%time
# setup model
from lib.model.model import *

from functools import partial
layer = partial( conv_layer1, stride=1, bn=False, zero_bn=False, act=GeneralRelu, dropout_ratio=0. )
mm    = CnnModelManager( model )
mm.initialize(is_resnet=False)


#xb,_ = mm.getFirstbatch( databunch, normalization = tensor_input_view)
#print(f"input shape:{xb.shape}")
#mm.summary(xb, print_mod=False)

# We start by a regressionmodel that predict the next OHLC
 - First try will be the to minimize the squared sum of errors.

In [ ]:
input = torch.randn(1,2)
output = input.clone()
#print(input)
output[0,0] = -output[0,0]
output[0,1] = -output[0,1]
#print(output)
def accuracy_sign(target, actual): 
    print(target)
    print(actual)
    return (torch.sgn(target).eq(torch.sgn(actual))).float().mean()
accuracy_sign(input, output)

In [ ]:
#def my_mse_loss(t_in, t_target): return (t_in-t_target).square().mean()
def accuracy_sign(out, yb): return (torch.sgn(out).eq(torch.sgn(yb))).float().mean()
sched        = combine_scheds([0.3, 0.7], [sched_cos(0.3, 0.6), sched_cos(0.6, 0.2)]) 
opt       = SGD(sched,max_lr=0.2)
loss_func = torch.nn.MSELoss() #torch.nn.SmoothL1Loss(), my_mse_loss 

sched        = combine_scheds([0.3, 0.7], [sched_cos(0.3, 0.6), sched_cos(0.6, 0.2)]) 
cbfs         = [TrainableModelCallback, TrainEvalCallback, OptimizerCallback, 
                partial(BatchTransformXCallback, tfm = tensor_input_view), 
                Recorder, 
                partial(AvgStatsCallback,[torch.nn.functional.mse_loss, accuracy_sign]),
                ProgressCallback
               ]

learn = Learner( mm.model, databunch, loss_func=loss_func)
%time learn.fit(10, opt=opt, cb_funcs=cbfs)

In [ ]:
#0.000003
#0.000002

#0.000163
#0.000286
#0.000095


#0.000284	
#0.000082
#0.000056
learn.find_subcription_by_cls(Recorder).plot_lr(),plt.show()
learn.find_subcription_by_cls(Recorder).plot_loss(skip_start=0),plt.show()

In [ ]:
%%time
"""
def predict_stocks(dataset, modelmanager, stocks, column_names, tfm_input ):
    price_sequences, price_targets = dataset.getBatch(stocks=stocks)
    #print(price_sequences)
    predictions    = modelmanager.predict(price_sequences, tfm_input)

    prediction_column_names = [ predict_prefix + name for name in ohlc_ds.column_names ]
    for idx,name in enumerate(prediction_column_names):
        price_targets.insert(len(price_targets.columns), name, predictions[:,idx].numpy())
    return price_targets, prediction_column_names
"""
#price_targets, prediction_column_names = predict_stocks(test_ds, mm, test_ds.stocks, tensor_input_view )

stocks = ["A","B"]
price_targets, prediction_column_names = predict_stocks(test_ds, mm, stocks, tensor_input_view )
cols = test_ds.column_names.copy()
cols.extend(training_column_names)    
price_targets[cols]

In [ ]:
#statistics on the mature stocks
from lib.finance.data import *
"""
predict_prefix = "predict_"
[PREDICT_OPEN,PREDICT_HIGH,PREDICT_LOW,PREDICT_CLOSE] = [predict_prefix+OPEN,\
                                                         predict_prefix+HIGH,\
                                                         predict_prefix+LOW,\
                                                         predict_prefix+CLOSE]
print(OPEN,PREDICT_OPEN)
"""
columns = training_column_names.copy()
columns.extend([predict_prefix + c for c in training_column_names])
price_targets.loc[validStocks,columns].describe(percentiles=[0.0002, 0.25, 0.75, 0.9998])        
#price_targets.loc[validStocks,[OPEN,PREDICT_OPEN]].describe(percentiles=[0.0002, 0.25, 0.75, 0.9998])        


In [ ]:
predict_prefix="predict_"
PREDICT_OPEN  = predict_prefix+OPEN
PREDICT_HIGH  = predict_prefix+HIGH
PREDICT_LOW   = predict_prefix+LOW
PREDICT_CLOSE = predict_prefix+CLOSE

In [ ]:
#plot signal vs prediction
#AAPL
from lib.finance.lineplot import *
stock = "A"
df         = price_targets.loc[stock].iloc[:256] # extract a section
dates      = df.loc[stock].index
reference  = (df.loc[stock,OPEN],stock+" reference","blue","line")
prediction = (df.loc[stock,PREDICT_OPEN],stock+" prediction","orange","circle")
plotSignalVSPrediction(dates,[reference,prediction])

In [ ]:
#mm.model[0][0].weight
#mm.model[0][0].bias

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()